In [34]:
#from google.colab import drive
#drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2
%autosave 180

import sys
#sys.path.append("/content/drive/MyDrive/137-codes/assignment3-solution")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Convolutional Neural Networks

**Objective:** This assignment aims to help you understand CNNs through implementing layers for a CNN network. It also provide you a chance to tune a CNN. 


**Tasks:** The first part of the assignment asks you to implement three types of layers (batch normalization, convolution, and pooling) that are commonly used in CNNs. To reduce the level of difficulty, you can implement convolution and pooling layers with `numpy`, and you are allowed to use for-loops. Your calculation will be compared against `torch` functions. 

The second part of the assignment asks you to construct and train a convolutional neural network on the **bean-leaf dataset ([link](https://www.kaggle.com/datasets/prakharrastogi534/bean-leaf-dataset))**. In this part work, you are expected to construct a convolutional neural network using **`torch` layers**. Then you apply the CNN to the dataset. You need to tune the model and try to get the highest test accuracy. 


**Detailed Instructions:**  Please implement all functions in `my_layers.py` and run through tests of your layers in this ipython notebook. 

In the classifcation task, you need to construct a CNN in `conv_net.py`. Then you should train the CNN you have implemented. You may want to tune your CNN through multiple runs of training and validation to find a good model.  

At the end, you need to save your model to this folder for submission. We will test your saved model to decide your points for the last part of work. 


**Grading:** Please finish your work in the folder `assignment3`. If you use google colab, please upload folder `assignment3` to your google drive and run your code there. When you have finished your work, please print your notebook to a pdf file and put it to `assignment3`. You need to submit the entire `assignment3` folder, which should contain:  
* all your implementations;
* the notebook that can run with your implementations; 
* your saved model; and
* a pdf print of your notebook. 

(If Gradescope does not accept your submission due to size limit, please let us know).

When we grade your code, we check your result notebook as well as your code. If your code cannot generate the result of a problem in your notebook file, you will get zero point for that problem. 

**GPU resources:** In this assignment you will need to use GPU to tune your model for the bean-leaf task. You can use GPU from google colab and Kaggle. You have up to 30 GPU hours/week from Kaggle ([link](https://www.kaggle.com/general/108481)). Google colab does not have an anounced limit, but you should be able to get more hours than Kaggle. Here are some tips for GPU usage. 

1. Only use GPU when necessary. You may want to garantee that your model runs before you put it to GPU.
2. Start your work early, so you have more GPU time
3. Remember to release GPU when you don't run your model. 

**Deadline:** Oct 27, 23:59pm, 2023. 

In [35]:
# As usual, a bit of setup
import numpy as np
import torch
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython


def mean_diff(x, y):
    with torch.no_grad():
        if isinstance(x, torch.Tensor):
            x = x.numpy()
        if isinstance(y, torch.Tensor):
            y = y.numpy()
        err = np.mean(np.abs(x - y))
    return err

# If you have cuda, do the following
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Batch normalization
**Question 1 (4 points).** Please implement the batch normalization layer in `bn_layer.py`. The following code block will test your implementation against the `torch` implementation. Please refer to the documentation of `torch.nn.BatchNorm2d`([link](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)) for the detailed calculation of batch normalization. 

In [36]:
from bn_layer import BNLayer
from torch.nn import BatchNorm2d
# initialize a tensor X and pretend that the tensor is a feature map in a CNN.
# In this case, we do normalization over different channels. As we have mentioned in the
# class, all pixels in a channel share the same normalization parameters.

np.random.seed(137)
N, C, H, W = 200, 5, 50, 60
X = torch.tensor(np.random.randn(N, C, H, W) * 0.2 + 5, dtype = torch.float32)

# initialize parameters for batch normalization

momentum = 0.1
epsilon = 0.01


# batch_normalization with torch.

torch_bn = BatchNorm2d(5, eps = epsilon, momentum = momentum, dtype = torch.float32)

# we use the same parameters as torch batch normalization.
my_bn = BNLayer(5, eps=epsilon, momentum=momentum)


# compare computation results over different training batches.

res_tf = torch_bn(X[0:10])
res_my = my_bn(X[0:10])


print("Calculation from the first batch: the difference between the two implementations is ",
          mean_diff(res_tf, res_my))

# run batch normalization on the second batch

res_tf = torch_bn(X[10:20])
res_my = my_bn(X[10:20])

print("Calculation from the second batch: the difference between the two implementations is ",
          mean_diff(res_tf, res_my))

# run batch normalization on the third batch

res_tf = torch_bn(X[20:30])
res_my = my_bn(X[20:30])

print("Calculation from the third batch: the difference between the two implementations is ",
          mean_diff(res_tf, res_my))

# compare computation results over a testing batch
my_bn.eval()
torch_bn.eval()

res_tf = torch_bn(X[40:50])
res_my = my_bn(X[40:50])
print("Calculation from testing: the difference between the two implementations is ",
       mean_diff(res_tf, res_my))


Calculation from the first batch: the difference between the two implementations is  9.53356e-06
Calculation from the second batch: the difference between the two implementations is  9.58232e-06
Calculation from the third batch: the difference between the two implementations is  9.637566e-06
Calculation from testing: the difference between the two implementations is  1.9003392e-07


## The convolution operation

**Question 2 (4 points).** Please implement a convolutional operation in `my_layers.py`. The following code block will test your implementation against the corresponding `torch` function. Please refer to the documentation of `torch.nn.Conv2d` for the detailed calculation of the convolution operation. 

**Note:** You only need to consider `stride = 1`, `padding` is `same` or `valid`. 

In [37]:
from my_layers import conv_forward
import torch.nn as nn

# initialize a tensor in the NCHW format and pretend the tensor is a feature map in a CNN.
x = np.random.randn(*[1, 5, 10, 20]).astype(np.float32)


# The first test case:
h_stride = 1
w_stride = 1
padding = "same"
t_mod = nn.Conv2d(in_channels=5, out_channels = 4, kernel_size = (3,3), stride=[h_stride, w_stride], padding=padding)
t_out = t_mod(torch.tensor(x, dtype = torch.float32))
with torch.no_grad():
    w = t_mod.weight.numpy()
    b = t_mod.bias.numpy()
out = conv_forward(input=x, filters=w, bias = b, stride=[h_stride, w_stride], padding=padding)

with torch.no_grad():
    print('Difference between your implementation and torch calculation:', mean_diff(out, t_out))

# The second test case:
h_stride = 1
w_stride = 1
padding = "same"
t_mod = nn.Conv2d(in_channels=5, out_channels = 4, kernel_size = (7,7), stride=[h_stride, w_stride], padding=padding)
t_out = t_mod(torch.tensor(x, dtype = torch.float32))
with torch.no_grad():
    w = t_mod.weight.numpy()
    b = t_mod.bias.numpy()
out = conv_forward(input=x, filters=w, bias = b, stride=[h_stride, w_stride], padding=padding)

with torch.no_grad():
    print('Difference between your implementation and torch calculation:', mean_diff(out, t_out))


# The third test case:
h_stride = 1
w_stride = 1
padding = "valid"
t_mod = nn.Conv2d(in_channels=5, out_channels = 4, kernel_size = (7,3), stride=[h_stride, w_stride], padding=padding)
t_out = t_mod(torch.tensor(x, dtype = torch.float32))
with torch.no_grad():
    w = t_mod.weight.numpy()
    b = t_mod.bias.numpy()
out = conv_forward(input=x, filters=w, bias = b, stride=[h_stride, w_stride], padding=padding)

with torch.no_grad():
    print('Difference between your implementation and torch calculation:', mean_diff(out, t_out))


# The third test case:
w = np.random.randn(3, 3, 5, 4).astype(np.float32)
h_stride = 1
w_stride = 1
padding = "valid"

t_mod = nn.Conv2d(in_channels=5, out_channels = 4, kernel_size = (3, 3), stride=[h_stride, w_stride], padding=padding)
t_out = t_mod(torch.tensor(x, dtype = torch.float32))
with torch.no_grad():
    w = t_mod.weight.numpy()
    b = t_mod.bias.numpy()
out = conv_forward(input=x, filters=w, bias = b, stride=[h_stride, w_stride], padding=padding)

with torch.no_grad():
    print('Difference between your implementation and torch calculation:', mean_diff(out, t_out))

Difference between your implementation and torch calculation: 5.0073795136995615e-08
Difference between your implementation and torch calculation: 9.3274429673329e-08
Difference between your implementation and torch calculation: 9.455996203339762e-08
Difference between your implementation and torch calculation: 5.9219827461573814e-08


# The max-pooling operation

**Question 4 (4 points).** Please implement the forward pass for the max-pooling operation in the function `pooling_forward_naive` in the file `my_layers.py`. Your implementation will be compared against torch implementation. 

In [38]:
from my_layers import max_pool_forward

np.random.seed(137)
# shape is NCHW
x = np.random.randn(2, 4, 10, 20).astype(np.float32)

pool_height = 3
pool_width = 3
stride_h = pool_height
stride_w = pool_width

my_out = max_pool_forward(x, [pool_height, pool_width], stride=[stride_h, stride_w], padding='VALID')
t_mod = nn.MaxPool2d([pool_height, pool_width], stride=[stride_h, stride_w])
t_out = t_mod(torch.tensor(x, dtype = torch.float32))

with torch.no_grad():
    t_out = t_mod(torch.tensor(x, dtype = torch.float32))
    print('The difference between your implementation and torch operation is: ', mean_diff(my_out, t_out))



pool_height = 4
pool_width = 4
stride_h = 2
stride_w = 2
my_out = max_pool_forward(x, [pool_height, pool_width], stride=[stride_h, stride_w], padding='VALID')
t_mod = nn.MaxPool2d([pool_height, pool_width], stride=[stride_h, stride_w])
t_out = t_mod(torch.tensor(x, dtype = torch.float32))

with torch.no_grad():
    t_out = t_mod(torch.tensor(x, dtype = torch.float32))
    print('The difference between your implementation and torch operation is: ', mean_diff(my_out, t_out))


pool_height = 5
pool_width = 4
stride_h = 2
stride_w = 1

my_out = max_pool_forward(x, [pool_height, pool_width], stride=[stride_h, stride_w], padding='VALID')
t_mod = nn.MaxPool2d([pool_height, pool_width], stride=[stride_h, stride_w])
with torch.no_grad():
    t_out = t_mod(torch.tensor(x, dtype = torch.float32))
    print('The difference between your implementation and torch operation is: ', mean_diff(my_out, t_out))


The difference between your implementation and torch operation is:  0.0
The difference between your implementation and torch operation is:  0.0
The difference between your implementation and torch operation is:  0.0


# Implement a CNN with Torch and train it

In this task, you need to build a CNN with `torch` layers and train it on the `beans` dataset. The `beans` dataset is an image classification task. The dataset contains images of bean leafs, and these images falls into three categories. The task is to predict the label of a leaf. 

**Question 5 (5 points).** Implementation: in your implementation of the neural network, you need to construct your CNN using existing `torch` layers. You CANNOT directly load a CNN such as ResNet from `torch`. You can check the example code provided by [chapter 8](https://d2l.ai/chapter_convolutional-modern/resnet.html) of D2L book.

NOTE: you CANNOT copy code from any resources.

**Question 6 (6 points).** Model tuning: depending the final performance of your trained model, you will get
* 2 points if the test accuracy is over 0.7
* 4 points if the test accuracy is over 0.8
* 6 points if the test accuracy is over 0.85


## Train the CNN


### Load Data

In [39]:
from torchvision.transforms import ToTensor, Compose
from torch.utils.data import random_split, DataLoader
import torch

In [40]:
from bean_dataset import BeanImageDataset

trainset = BeanImageDataset("data/train")
validset = BeanImageDataset("data/validation")

train_loader = DataLoader(trainset, batch_size=4, shuffle=True)
valid_loader = DataLoader(validset, batch_size=len(validset), shuffle=False)

### Train the model 

Then you need to train your network below. You are supposed to implementing the training loop in the notebook cell below. Please pay attention to TODOs. Feel free to change hyper-parameters. 

In [41]:
from conv_net import ConvNet, train
import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if device.type == "cuda":
    total_mem = torch.cuda.get_device_properties(0).total_memory
else:
    total_mem = 0

max_epoch = 100
learning_rate = 1e-3

# Load your CNN model
model = ConvNet()

# Train the CNN model
# the train function is supposed to return the average training loss,
# the average validation loss, and the validation accuracy in each epoch
# then we can plot the training behavior below.
train_loss, val_loss, val_acc = train(model, train_loader, valid_loader, learning_rate, max_epoch, device)

# The print-out lines are from my own training function, and you don't have to reproduce these lines.

Epoch [1]: Training Loss: 0.8650733423509193 Validation Loss: 0.8108940124511719 Accuracy: 79.0
Epoch [2]: Training Loss: 0.8408999518423007 Validation Loss: 0.7110354900360107 Accuracy: 93.0
Epoch [3]: Training Loss: 0.8521327337242922 Validation Loss: 0.660572350025177 Accuracy: 95.0
Epoch [4]: Training Loss: 0.8149366196978507 Validation Loss: 0.7012324929237366 Accuracy: 92.0
Epoch [5]: Training Loss: 0.8033484158368644 Validation Loss: 0.7202180027961731 Accuracy: 93.0
Epoch [6]: Training Loss: 0.7587133870967107 Validation Loss: 0.6025179624557495 Accuracy: 98.0
Epoch [7]: Training Loss: 0.7459939943202214 Validation Loss: 0.706694483757019 Accuracy: 91.0
Epoch [8]: Training Loss: 0.7386378671211625 Validation Loss: 0.7544174194335938 Accuracy: 88.0
Epoch [9]: Training Loss: 0.7570302328202715 Validation Loss: 0.6310612559318542 Accuracy: 99.0
Epoch [10]: Training Loss: 0.7319438069425955 Validation Loss: 0.5605678558349609 Accuracy: 100.0
Epoch [11]: Training Loss: 0.72651319838

KeyboardInterrupt: 

### Plot the training history

One important way of diagnosing model training is to plot training and validation losses during the training procedure. In general, we should see a decreasing of training and validation losses. With my naive CNN model and only 4 epochs, the validation loss is even smaller than training loss, which is somewhat unusual, but a typical model should get higher validation losses.   

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_loss, '-o')
plt.plot(val_loss, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(val_acc, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

### Save the model

In [ ]:
torch.save(model, "bean_classifier.sav")

### Test the saved model on a test set

In [ ]:
# Load the previously saved model.
testset = BeanImageDataset("data/test")
test_loader = DataLoader(testset, batch_size=1, shuffle=True)

model = torch.load("bean_classifier.sav")
model.eval()


t_acc = []
with torch.no_grad():
    # only one item in the iterator
    # Add more batches if your device couldn't handle the computation
    for _, data in enumerate(test_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        y_hat = model(x)

        if device.type == "cuda":
          x = x.to("cpu")
          y = y.to("cpu")
          y_hat = y_hat.to("cpu")
        acc = np.average(y.numpy() == np.argmax(y_hat.numpy(), axis = 1))
        t_acc.append(acc.item())
print(f"Test accuracy: {np.average(t_acc)}")

Test accuracy: 0.8671875
